# Load data

In [2]:
import pandas as pd
import chromadb as chromadb

In [3]:
# data source: https://www.kaggle.com/datasets/nulldata/medium-post-titles
df = pd.read_csv("medium_post_titles.csv")

df = df.dropna()
df = df[~df["subtitle_truncated_flag"]]

topics_of_interest = ['artificial-intelligence', 'data-science', 'machine-learning']
# topics_of_interest = ['data-science']

df = df[df['category'].isin(topics_of_interest)]

df['text'] = df['title']  + df['subtitle']

df['meta'] = df.apply( lambda x: {
    'text': x['text'],
    'category': x['category']  
}, axis=1)

In [5]:
df.head(2)

,category,title,subtitle,subtitle_truncated_flag,text,meta
4,artificial-intelligence,"""Can I Train my Model on Your Computer?""",How we waste computational resources and how t...,False,"""Can I Train my Model on Your Computer?""How we...","{'text': '""Can I Train my Model on Your Comput..."
289,data-science,(Robot) data scientists as a service,Automating data science with symbolic regressi...,False,(Robot) data scientists as a serviceAutomating...,{'text': '(Robot) data scientists as a service...


# chroma db setup

In [27]:
from chromadb.config import Settings

In [7]:
# initiating a client
chroma_client = chromadb.Client() #in memory db
# collections(its like an abstract index)
article_collection=chroma_client.create_collection(name='medium-article')

In [29]:
# initiating a client
chroma_client = chromadb.PersistentClient(
    path="medium-chroma-db"
) #persistent memory
# collections(its like an abstract index)
article_collection=chroma_client.create_collection(name='medium-article')

In [18]:
## data insertion

In [30]:
# inserting data
article_collection.upsert(
    ids=[f"{x}" for x in df.index.tolist()],
    documents=df['text'].tolist(),
    metadatas=df['meta'].tolist()
)

In [19]:
## [f"{x}" for x in df.index] list (int to str)

# vector query

In [25]:
qry_str="best data ai library?"

In [26]:
article_collection.query(query_texts=qry_str,n_results=1)

{'ids': [['103719']],
 'embeddings': None,
 'documents': [['Top 7 libraries and packages of the year for Data Science and AI: Python & RThis is a list of the best libraries and packages that changed our lives this year, compiled from my weekly digests']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'text': 'Top 7 libraries and packages of the year for Data Science and AI: Python & RThis is a list of the best libraries and packages that changed our lives this year, compiled from my weekly digests',
    'category': 'machine-learning'}]],
 'distances': [[0.6297022700309753]]}